In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import seaborn as sns
import os
import folium
from folium.plugins import MarkerCluster



In [35]:
maps = os.listdir("ml/data/map")
maps.remove("us_map.npy")
maps_dict = {map.replace(".npy",""): np.load(f"ml/data/map/{map}") for map in maps}
xmap = np.load("ml/data/map/us_map.npy")[::-1]

In [36]:
clean = os.listdir("data/clean")
clean_dict = {file.replace(".csv",""): pd.read_csv(f"data/clean/{file}") for file in clean}
clean = list(clean_dict.keys())

In [37]:
cities = pd.read_csv("american_cities.csv").drop(columns=["Unnamed: 0"])
cities = cities.drop_duplicates(keep="first", subset=["city"])
cities.set_index("city", inplace=True)

In [27]:
CITY = {"name": "Kansas City"}
CITY["lat"] = cities.loc[CITY["name"]]["lat"]
CITY["lon"] = cities.loc[CITY["name"]]["lng"]

colors = ["#f8b195", "#f67280", "#6C5B7B", "#355C7D"]

In [32]:
if CITY["name"] == "San Francisco":
    CITY["components"] = [clean_dict["elec_stations_clean"], clean_dict["docks_clean"], clean_dict["airports_clean"]]
    CITY["components_name"] = ["Borne de recharge électrique", "Port", "Aéroports"]
    CITY["area radius"] = 100

if CITY["name"] == "New York":
    CITY["components"] = [clean_dict["elec_stations_clean"], clean_dict["docks_clean"], clean_dict["airports_clean"]]
    CITY["components_name"] = ["Borne de recharge électrique", "Port", "Aéroports"]
    CITY["area radius"] = 1000

if CITY["name"] == "Kansas City":
    CITY["components"] = [clean_dict["lpg_stations_clean"], clean_dict["e85_stations_clean"]]
    CITY["components_name"] = ["Stations de gaz de pétrole liquéfié", "Stations superéthanol e85"]
    CITY["area radius"] = 50

In [33]:
item_txt = """<br> &nbsp; {item} &nbsp; <i class="fa fa-circle fa-1.5x" style="color:{col}"></i> \n"""

html_itms = ""
for i, name in enumerate(CITY["components_name"]):
    html_itms += item_txt.format( item=name , col=colors[i])

legend_html = """
     <div style="
     position: fixed; 
     bottom: 50px; left: 50px; width: 500px; height: 160px; 
     border:2px solid grey; z-index:9999; 
     
     background-color:white;
     opacity: .85;
     
     font-size:24px;
     font-weight: bold;
     
     ">
     &nbsp; <u>{title}</u> 
     
     {itm_txt}

      </div> """.format( title = "Légende" , itm_txt= html_itms)

map = folium.Map(location=[CITY["lat"], CITY["lon"]], zoom_start=8,max_bounds=True, min_zoom=8, tiles="cartodbpositron")

lat_bounds = 4
lon_bounds = 4

cities = cities.loc[(cities["lat"]>CITY["lat"]-lat_bounds) & (cities["lat"]<CITY["lat"]+lat_bounds) & (cities["lng"]>CITY["lon"]-lon_bounds) & (cities["lng"]<CITY["lon"]+lon_bounds)][:10]
for city in cities.index:
    folium.Circle([cities.loc[city]["lat"], cities.loc[city]["lng"]], radius=cities.loc[city]["population"]/CITY["area radius"], popup=f"{city} population: {cities.loc[city]['population']}", color="black", fill=True,
 fill_color="black", opacity=0.2).add_to(map)
 
#diplay points on map
for ind, component in enumerate(CITY["components"]):
    
    for index, row in component.loc[(component["latitude"]>CITY["lat"]-lat_bounds) & (component["latitude"]<CITY["lat"]+lat_bounds) & (component["longitude"]>CITY["lon"]-lon_bounds) & (component["longitude"]<CITY["lon"]+lon_bounds)].iterrows():
        folium.CircleMarker([row["latitude"], row["longitude"]], radius=1, color=colors[ind], fill=True, fill_opacity=1).add_to(map)

folium.LayerControl().add_to(map)
map.get_root().html.add_child(folium.Element( legend_html ))
map.save("maps_html/kansascity.html")

In [214]:
for val in cities.iterrows():
    print(type(val))
    break

<class 'tuple'>


In [215]:
cities.loc["Portland"]

lat                 45.5372
lng                 -122.65
country       United States
population          2052796
Name: Portland, dtype: object

In [216]:
clean_dict["elec_stations_clean"].shape

(56740, 3)

In [41]:
clean_dict["lpg_stations_clean"]

,longitude,latitude,val
0,-88.522923,42.660919,1
1,-101.885391,35.267353,1
2,-68.009825,46.842162,1
3,-73.429554,41.082789,1
4,-74.232109,40.782618,1
...,...,...,...
2747,-82.736230,35.233880,1
2748,-79.880770,36.949560,1
2749,-96.211889,42.766956,1
2750,-95.858110,36.076870,1
